In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.options.plotting.backend = "plotly"

# Rutas de archivos
PROCESSED_DATA_DIR = '../data/processed'
FILE_2000_2011 = os.path.join(PROCESSED_DATA_DIR, 'fondef_2000_2011_processed.csv')
FILE_2012_2017 = os.path.join(PROCESSED_DATA_DIR, 'fondef_2012_2017_processed.csv')

## 3. Carga y Preparación de Datos

Cargamos los datasets procesados previamente por nuestro pipeline ETL.

In [2]:
# Cargar datos
df_2000 = pd.read_csv(FILE_2000_2011)
df_2012 = pd.read_csv(FILE_2012_2017)

print(f"Registros 2000-2011: {df_2000.shape}")
print(f"Registros 2012-2017: {df_2012.shape}")

Registros 2000-2011: (592, 9)
Registros 2012-2017: (508, 9)


### 3.1 Ingeniería de Características: Extracción de Año para Dataset 2012-2017

El dataset de 2012-2017 no contenía una columna explícita de año en el origen. Sin embargo, podemos inferir el año a partir del código del proyecto (ej. `ID14...` corresponde a 2014, `CA12...` a 2012).

Aplicaremos una lógica de extracción para enriquecer la data.

In [3]:
def extract_year_from_code(code):
    """
    Extrae el año del código del proyecto.
    Asume formato XXYY... donde YY es el año (12, 13, 14, etc.)
    """
    try:
        if pd.isna(code):
            return np.nan
        
        # Extraer los dígitos en la posición 2 y 3 (índices 2:4)
        # Ej: ID14I10335 -> '14'
        # Ej: CA12I10022 -> '12'
        year_str = str(code)[2:4]
        
        if year_str.isdigit():
            year = int(year_str)
            # Ajustar a formato 20YY
            return 2000 + year
        return np.nan
    except:
        return np.nan

# Aplicar extracción si el año es 0 o nulo
mask_invalid_year = (df_2012['year'] == 0) | (df_2012['year'].isna())
df_2012.loc[mask_invalid_year, 'year'] = df_2012.loc[mask_invalid_year, 'project_code'].apply(extract_year_from_code)

print("Distribución de años recuperados en dataset 2012-2017:")
print(df_2012['year'].value_counts().sort_index())

Distribución de años recuperados en dataset 2012-2017:
year
2012     55
2013     93
2014     71
2015     98
2016    116
2017     75
Name: count, dtype: int64


## 4. Análisis Visual: Montos por Área y Año

A continuación, generamos visualizaciones interactivas para responder a la pregunta de negocio.

### 4.1 Evolución Temporal de Montos Totales

In [5]:
# Agrupar por año
yearly_amounts = df_clean.groupby('year')['amount'].sum().reset_index()

fig = px.bar(yearly_amounts, x='year', y='amount',
             title='Evolución Total de Montos Adjudicados por Año',
             labels={'amount': 'Monto Total (Miles $)', 'year': 'Año'},
             text_auto='.2s',
             template='plotly_white')

fig.update_traces(marker_color='rgb(55, 83, 109)', marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(xaxis_tickmode='linear')
fig.show()

### 4.2 Montos por Área y Año (Heatmap)

Esta visualización permite identificar rápidamente qué áreas recibieron más financiamiento en qué años.

In [6]:
# Pivot table para heatmap
pivot_area_year = df_clean.pivot_table(values='amount', index='area', columns='year', aggfunc='sum', fill_value=0)

fig = px.imshow(pivot_area_year,
                labels=dict(x="Año", y="Área", color="Monto Adjudicado"),
                x=pivot_area_year.columns,
                y=pivot_area_year.index,
                title="Mapa de Calor: Financiamiento por Área y Año",
                aspect="auto",
                color_continuous_scale="Viridis")

fig.update_layout(xaxis_tickmode='linear')
fig.show()

### 4.3 Detalle Interactivo: Montos por Área a través del tiempo

Un gráfico de líneas interactivo para comparar la tendencia de diferentes áreas.

In [7]:
# Agrupar por año y área
area_year_amounts = df_clean.groupby(['year', 'area'])['amount'].sum().reset_index()

# Filtrar áreas con muy pocos fondos para no saturar el gráfico (Top 10 áreas históricas)
top_areas = df_clean.groupby('area')['amount'].sum().nlargest(10).index
df_top_areas = area_year_amounts[area_year_amounts['area'].isin(top_areas)]

fig = px.line(df_top_areas, x="year", y="amount", color="area",
              title="Tendencia de Financiamiento por Área (Top 10 Áreas)",
              labels={'amount': 'Monto (Miles $)', 'year': 'Año', 'area': 'Área'},
              markers=True,
              template='plotly_white')

fig.update_layout(xaxis_tickmode='linear', hovermode="x unified")
fig.show()

In [ ]:
# Gráfico de Barras Apiladas: Montos por Área y Año
# Esta visualización responde directamente a "Montos x Área x Año"
fig = px.bar(df_with_amounts, x="year", y="amount", color="area",
             title="Distribución de Montos Adjudicados por Área y Año (2000-2011)",
             labels={'amount': 'Monto (Miles $)', 'year': 'Año', 'area': 'Área'},
             template='plotly_white')

fig.update_layout(xaxis_tickmode='linear')
fig.show()

### 4.4 Distribución de Montos por Área (Treemap)

Visión jerárquica de cómo se distribuye el presupuesto total histórico.

In [ ]:
# Pivot table para conteo de proyectos (Todo el dataset)
pivot_count = df_clean.pivot_table(values='project_code', index='area', columns='year', aggfunc='count', fill_value=0)

fig = px.imshow(pivot_count,
                labels=dict(x="Año", y="Área", color="Cantidad de Proyectos"),
                x=pivot_count.columns,
                y=pivot_count.index,
                title="Mapa de Calor: Cantidad de Proyectos por Área y Año (2000-2017)",
                aspect="auto",
                color_continuous_scale="Magma")

fig.update_layout(xaxis_tickmode='linear')
fig.show()

### 4.3 Cantidad de Proyectos por Área y Año (Todo el periodo)

Dado que no tenemos montos para el segundo periodo, analizamos la cantidad de proyectos adjudicados, lo cual nos da una proxy de la actividad en cada área.

In [8]:
# Agrupar por área para el total histórico
area_total = df_clean.groupby('area')['amount'].sum().reset_index()

fig = px.treemap(area_total, path=['area'], values='amount',
                 title='Distribución Total de Fondos por Área (Histórico)',
                 color='amount',
                 color_continuous_scale='RdBu',
                 hover_data=['amount'])

fig.update_traces(textinfo="label+value+percent root")
fig.show()

## 5. Conclusiones

Basado en el análisis visual:

1.  **Tendencias Temporales**: Se puede observar la evolución del presupuesto disponible. (Analizar picos y valles en el gráfico 4.1).
2.  **Áreas Dominantes**: El Treemap (4.4) revela claramente qué sectores han sido prioritarios para Fondef/IDeA en el periodo analizado.
3.  **Dinámica por Área**: El gráfico de líneas (4.3) permite ver si ciertas áreas han ganado o perdido relevancia con los años.

Este notebook sirve como base para análisis más profundos, como la minería de texto en los títulos de proyectos o la correlación entre palabras clave y montos adjudicados.